In [1]:
%matplotlib inline
import os
import sys

#uncomment if cloned from github!
sys.path.append("../")

from sapsan.lib.backends.fake import FakeBackend
from sapsan.lib.backends.mlflow import MLflowBackend
from sapsan.lib.data.hdf5_dataset import HDF5Dataset
from sapsan.lib.data import EquidistanceSampling
from sapsan.lib.estimator import CNN3d, CNN3dConfig
from sapsan.lib.experiments.evaluate_3d import Evaluate3d
from sapsan.lib.experiments.train import Train

/opt/anaconda/envs/sapsan/lib/python3.7/site-packages/catalyst/tools/registry.py:45: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class Registry(collections.MutableMapping):
/opt/anaconda/envs/sapsan/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
#--- Experiment tracking backend ---
experiment_name = "CNN experiment"

#Fake (disable backend)
tracking_backend = FakeBackend(experiment_name)

#MLflow
#launch mlflow with: mlflow ui --port=9000
#uncomment tracking_backend to use mlflow

MLFLOW_HOST = "localhost"
MLFLOW_PORT = 9000

#tracking_backend = MLflowBackend(experiment_name, MLFLOW_HOST, MLFLOW_PORT)

/opt/anaconda/envs/sapsan/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [3]:
#--- Data setup ---
#In the intereset of loading and training multiple timesteps
#one can specify which checkpoints to use and where
#they appear in the path via syntax: {checkpoint:format}
#
#Next, you need to specify which features to load; let's assume 
#        path = "{feature}.h5"
#
# 1) If in different files, then specify features directly;
#    The default HDF5 label will be the last label in the file
#    Ex: features = ['velocity', 'denisty', 'pressure']
# 2) If in the same, then duplicate the name in features
#    and specify which labels to pull
#    Ex: features = ["data", "data", "data"]
#        feature_labels = ['velocity', 'density', 'pressure']

path = "data/t{checkpoint:1.0f}/{feature}_dim32_fm15.h5"
features = ['u']
target = ['u']

#dimensionality of the data
AXIS = 3

#Dimensionality of your data per axis
CHECKPOINT_DATA_SIZE = 32

#Reduce dimensionality of each axis to
SAMPLE_TO = 16

#Dimensionality of each axis in a batch
GRID_SIZE = 8

#Sampler to use for reduction
sampler = EquidistanceSampling(CHECKPOINT_DATA_SIZE, SAMPLE_TO, AXIS)

In [4]:
#--- Train the model ---
#Machine Learning model to use

#Configuration of the model parameters:
#    n_epochs = number of epochs (iterations)
#    grid_dim = size of the batch
#    patience = number of epochs to run beyond convergence
#    min_delta = loss based convergence cut-off
estimator = CNN3d(
    config=CNN3dConfig(n_epochs=1, grid_dim=GRID_SIZE, patience=10, min_delta=1e-5)
)

#Load the data
data_loader = HDF5Dataset(path=path,
                   features=features,
                   target=target,
                   checkpoints=[0],
                   grid_size=GRID_SIZE,
                   checkpoint_data_size=CHECKPOINT_DATA_SIZE,
                   sampler=sampler)
x, y = data_loader.load()

#Set the experiment
training_experiment = Train(name=experiment_name,
                             backend=tracking_backend,
                             model=estimator,
                             inputs=x, targets=y,
                             data_parameters = data_loader.get_parameters())

#Train the model
training_experiment.run()

Loading 'u00000' from file 'data/t0/u_dim32_fm15.h5'
Loading 'u00000' from file 'data/t0/u_dim32_fm15.h5'


NameError: name 'np' is not defined

In [ ]:
#--- Test the model ---
#Load the test data
data_loader = HDF5Dataset(path=path,
                   features=features,
                   target=target,
                   checkpoints=[0],
                   grid_size=GRID_SIZE,
                   checkpoint_data_size=CHECKPOINT_DATA_SIZE,
                   sampler=sampler)
x, y = data_loader.load()

#Set the test experiment
evaluation_experiment = Evaluate3d(name=experiment_name,
                                   backend=tracking_backend,
                                   model=training_experiment.model,
                                   inputs=x, targets=y,
                                   data_parameters = data_loader.get_parameters())


#Test the model
evaluation_experiment.run()